In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import seaborn as sns; sns.set(style="ticks", palette='Set1')
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import scikitplot as skplt
import seaborn as sns

In [36]:
train = pd.read_csv('train.csv', delimiter=',', header = 0)
test = pd.read_csv('test.csv', delimiter=',', header = 0)
user_features = pd.read_csv('user-features.csv', delimiter = ',', header = 0)
item_features = pd.read_csv('item-features.csv', delimiter=',', header = 0)
test_ids = test['user_id'].tolist()

In [3]:
'''train['timestamp']=train['timestamp'].apply(datetime.datetime.fromtimestamp)
test['timestamp']=test['timestamp'].apply(datetime.datetime.fromtimestamp)

train.head(5)'''

"train['timestamp']=train['timestamp'].apply(datetime.datetime.fromtimestamp)\ntest['timestamp']=test['timestamp'].apply(datetime.datetime.fromtimestamp)\n\ntrain.head(5)"

In [4]:
#train["hour"] = train.timestamp.apply(lambda x: x.hour)
#train["weekday"] = train.timestamp.apply(lambda x: x.weekday())
#train['is_weekend'] = train['weekday'].isin([5,6])*1

#test["hour"] = test.timestamp.apply(lambda x: x.hour)
#test["weekday"] = test.timestamp.apply(lambda x: x.weekday())
#test['is_weekend'] = test['weekday'].isin([5,6])*1

train.head()

,user_id,item_id,like,timestamp
0,140,342,0,1490936622
1,378,172,1,1490936628
2,150,182,0,1490936650
3,455,17,0,1490936704
4,350,409,0,1490936735


In [37]:
def code_mean(data, cat_feature, real_feature):
    return dict(data.groupby(cat_feature)[real_feature].mean())

code_mean(train, 'user_id', 'like')

train = train[train['item_id'] < 209]
item_features = item_features[item_features['item_id'] < 209]

In [4]:
#for i in range(5):
    #user1 = train[train['user_id'] == i*50].sort_values('timestamp')
    #print(user1.head(10))
    #ax = user1.plot.scatter(x = 'hour', y = 'item_id', c = 'like', colormap = 'viridis')
    #ax = user1.plot(x = 'hour', y = 'item_id')

In [38]:
n_users = user_features.shape[0]
n_items = item_features.shape[0]
train_arr = train.values
interactions = np.zeros(n_users*n_items).reshape(n_users, n_items)
for line in train_arr:
    if int(line[2]) == 1:
        interactions[line[0]][line[1]] = 1
    else:
        interactions[line[0]][line[1]] = -1

In [39]:
interactions

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [-1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [5]:
user_features.head(5)

,user_id,0,1,2,3,4,5,6,7,8,...,22,23,24,25,26,27,28,29,30,31
0,0,0.000695,-0.001573,-0.001470,0.002085,-0.000498,0.000685,0.000811,0.000666,-0.003031,...,-0.004196,-0.000698,0.001121,-0.001079,-0.001993,-0.001993,0.000422,-0.001168,-0.001168,0.000297
1,1,0.001204,-0.002725,-0.002546,0.003612,-0.000862,0.001187,0.001404,0.001154,-0.005251,...,-0.007268,-0.001209,0.001942,-0.001870,-0.003451,-0.003451,0.000732,-0.002023,-0.002023,0.000515
2,2,0.000491,-0.001112,-0.001039,0.001475,-0.000352,0.000484,0.000573,0.000471,-0.002144,...,-0.002967,-0.000494,0.000793,-0.000763,-0.001409,-0.001409,0.000299,-0.000826,-0.000826,0.000210
3,3,0.000777,-0.001759,-0.001643,0.002332,-0.000557,0.000766,0.000906,0.000745,-0.003389,...,-0.004691,-0.000781,0.001254,-0.001207,-0.002228,-0.002228,0.000472,-0.001306,-0.001306,0.000332
4,4,0.000695,-0.001573,-0.001470,0.002085,-0.000498,0.000685,0.000811,0.000666,-0.003031,...,-0.004196,-0.000698,0.001121,-0.001079,-0.001993,-0.001993,0.000422,-0.001168,-0.001168,0.000297


In [6]:
item_features.iloc[:,1:].head(5)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.005646,-0.012780,-0.011941,0.016942,-0.004044,0.005566,0.006587,0.005411,-0.024627,-0.024627,...,-0.034090,-0.005673,0.009111,-0.008770,-0.016189,-0.016189,0.003432,-0.009490,-0.009490,0.002416
1,0.002530,-0.005726,-0.005350,0.007591,-0.001812,0.002494,0.002951,0.002424,-0.011035,-0.011035,...,-0.015274,-0.002542,0.004082,-0.003929,-0.007254,-0.007254,0.001538,-0.004252,-0.004252,0.001082
2,0.001592,-0.003604,-0.003368,0.004778,-0.001141,0.001570,0.001858,0.001526,-0.006946,-0.006946,...,-0.009615,-0.001600,0.002570,-0.002473,-0.004566,-0.004566,0.000968,-0.002677,-0.002677,0.000681
3,0.002866,-0.006486,-0.006060,0.008598,-0.002052,0.002825,0.003343,0.002746,-0.012499,-0.012499,...,-0.017301,-0.002879,0.004624,-0.004451,-0.008216,-0.008216,0.001742,-0.004816,-0.004816,0.001226
5,0.002577,-0.005833,-0.005450,0.007733,-0.001846,0.002540,0.003006,0.002470,-0.011241,-0.011241,...,-0.015560,-0.002589,0.004158,-0.004003,-0.007389,-0.007389,0.001566,-0.004332,-0.004332,0.001103


In [7]:
'''scaler = StandardScaler()
user_features_scaled = scaler.fit_transform(user_features[user_features.columns[1:]])
item_features_scaled = scaler.fit_transform(item_features[item_features.columns[1:]])'''

#pca = PCA(n_components=1)
#user_features_pca = pd.DataFrame(pca.fit_transform(user_features_scaled), columns = ['0'])

#pca = PCA(n_components=1)
#item_features_pca = pd.DataFrame(pca.fit_transform(item_features_scaled), columns = ['0'])

item_features.iloc[:,1:] = item_features_pca
item_features = item_features.iloc[:,:2]
user_features.iloc[:,1:] = user_features_pca
user_features = user_features.iloc[:,:2]

NameError: name 'item_features_pca' is not defined

In [8]:
item_features.head(5)

,item_id,0,1,2,3,4,5,6,7,8,...,22,23,24,25,26,27,28,29,30,31
0,35,0.005646,-0.012780,-0.011941,0.016942,-0.004044,0.005566,0.006587,0.005411,-0.024627,...,-0.034090,-0.005673,0.009111,-0.008770,-0.016189,-0.016189,0.003432,-0.009490,-0.009490,0.002416
1,19,0.002530,-0.005726,-0.005350,0.007591,-0.001812,0.002494,0.002951,0.002424,-0.011035,...,-0.015274,-0.002542,0.004082,-0.003929,-0.007254,-0.007254,0.001538,-0.004252,-0.004252,0.001082
2,145,0.001592,-0.003604,-0.003368,0.004778,-0.001141,0.001570,0.001858,0.001526,-0.006946,...,-0.009615,-0.001600,0.002570,-0.002473,-0.004566,-0.004566,0.000968,-0.002677,-0.002677,0.000681
3,36,0.002866,-0.006486,-0.006060,0.008598,-0.002052,0.002825,0.003343,0.002746,-0.012499,...,-0.017301,-0.002879,0.004624,-0.004451,-0.008216,-0.008216,0.001742,-0.004816,-0.004816,0.001226
5,32,0.002577,-0.005833,-0.005450,0.007733,-0.001846,0.002540,0.003006,0.002470,-0.011241,...,-0.015560,-0.002589,0.004158,-0.004003,-0.007389,-0.007389,0.001566,-0.004332,-0.004332,0.001103


In [9]:
train = train.join(user_features.set_index('user_id'), on='user_id', how = 'outer').join(item_features.set_index('item_id'), on='item_id', how = 'outer', rsuffix='_item')

In [10]:
X_train = train.loc[:,[name for name in train.columns if name !='like']]
y_train = np.array(train.loc[:,'like'])
X_train_new = train.loc[:,[name for name in train.columns if name not in ['timestamp', 'like', 'user_id', 'item_id']]]
X_train_new.head(5)

,0,1,2,3,4,5,6,7,8,9,...,22_item,23_item,24_item,25_item,26_item,27_item,28_item,29_item,30_item,31_item
1,0.003204,-0.007252,-0.006776,0.009613,-0.002295,0.003158,0.003737,0.003070,-0.013974,-0.013974,...,-0.013597,-0.002263,0.003634,-0.003498,-0.006457,-0.006457,0.001369,-0.003785,-0.003785,0.000964
6746,0.002554,-0.005780,-0.005401,0.007662,-0.001829,0.002517,0.002979,0.002447,-0.011138,-0.011138,...,-0.013597,-0.002263,0.003634,-0.003498,-0.006457,-0.006457,0.001369,-0.003785,-0.003785,0.000964
7599,0.004649,-0.010523,-0.009833,0.013950,-0.003330,0.004583,0.005424,0.004455,-0.020279,-0.020279,...,-0.013597,-0.002263,0.003634,-0.003498,-0.006457,-0.006457,0.001369,-0.003785,-0.003785,0.000964
640,0.004023,-0.009105,-0.008507,0.012070,-0.002881,0.003965,0.004693,0.003855,-0.017546,-0.017546,...,-0.013597,-0.002263,0.003634,-0.003498,-0.006457,-0.006457,0.001369,-0.003785,-0.003785,0.000964
3793,0.004038,-0.009139,-0.008539,0.012115,-0.002892,0.003980,0.004710,0.003869,-0.017611,-0.017611,...,-0.013597,-0.002263,0.003634,-0.003498,-0.006457,-0.006457,0.001369,-0.003785,-0.003785,0.000964


In [11]:
test.head(5)

,user_id,timestamp
0,166,1490944431
1,26,1490957371
2,41,1490958147
3,286,1490971255
4,108,1490976836


In [12]:
zr1 = np.zeros(test.shape[0])
zr2 = np.zeros(item_features.shape[0])
test['join_col'] = zr1
item_features['join_col'] = zr2

In [13]:
test = test.join(user_features.set_index('user_id'), on='user_id', how = 'outer').join(item_features.set_index('join_col'), on='join_col', how = 'outer', rsuffix='_item')

In [14]:
X_test = test.loc[:,[name for name in test.columns if name !='join_col']].reset_index(drop = True)
X_test_new = test.loc[:,[name for name in test.columns if name not in ['timestamp','like', 'user_id', 'item_id','join_col']]].reset_index(drop = True) #
#old_cols = X_test_new.columns.tolist()
#new_cols = old_cols[:4] + ['item_id'] + [col for col in old_cols[4:] if col != 'item_id']
#X_test_new = X_test_new[new_cols]

In [15]:
X_test_new.head()

,0,1,2,3,4,5,6,7,8,9,...,22_item,23_item,24_item,25_item,26_item,27_item,28_item,29_item,30_item,31_item
0,0.002382,-0.005392,-0.005038,0.007148,-0.001706,0.002348,0.002779,0.002283,-0.010391,-0.010391,...,-0.034090,-0.005673,0.009111,-0.008770,-0.016189,-0.016189,0.003432,-0.009490,-0.009490,0.002416
1,0.002382,-0.005392,-0.005038,0.007148,-0.001706,0.002348,0.002779,0.002283,-0.010391,-0.010391,...,-0.015274,-0.002542,0.004082,-0.003929,-0.007254,-0.007254,0.001538,-0.004252,-0.004252,0.001082
2,0.002382,-0.005392,-0.005038,0.007148,-0.001706,0.002348,0.002779,0.002283,-0.010391,-0.010391,...,-0.009615,-0.001600,0.002570,-0.002473,-0.004566,-0.004566,0.000968,-0.002677,-0.002677,0.000681
3,0.002382,-0.005392,-0.005038,0.007148,-0.001706,0.002348,0.002779,0.002283,-0.010391,-0.010391,...,-0.017301,-0.002879,0.004624,-0.004451,-0.008216,-0.008216,0.001742,-0.004816,-0.004816,0.001226
4,0.002382,-0.005392,-0.005038,0.007148,-0.001706,0.002348,0.002779,0.002283,-0.010391,-0.010391,...,-0.015560,-0.002589,0.004158,-0.004003,-0.007389,-0.007389,0.001566,-0.004332,-0.004332,0.001103


In [16]:
from catboost import CatBoostClassifier, Pool, cv

from sklearn.model_selection import train_test_split

X_train_, X_validation, y_train_, y_validation = train_test_split(X_train_new, y_train, train_size=0.8, random_state=42)

In [17]:
#cat_feat = ['hour', 'user_id', 'item_id']

params = {
    #'iterations': 300,
    #'learning_rate': 0.1,
    #'random_seed': 42,
    'eval_metric': 'Accuracy',
    #'logging_level': 'Silent',
    #'reg_lambda': 10,
    'use_best_model': True
}
train_pool = Pool(X_train_, y_train_)
validate_pool = Pool(X_validation, y_validation)

model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=validate_pool)


print('Simple model validation accuracy: {:.4}'
      .format(accuracy_score(y_validation, model.predict(X_validation))))

Learning rate set to 0.045977
0:	learn: 0.8521720	test: 0.8466312	best: 0.8466312 (0)	total: 83.2ms	remaining: 1m 23s
1:	learn: 0.8526152	test: 0.8554965	best: 0.8554965 (1)	total: 105ms	remaining: 52.4s
2:	learn: 0.8515071	test: 0.8554965	best: 0.8554965 (1)	total: 168ms	remaining: 55.9s
3:	learn: 0.8492908	test: 0.8590426	best: 0.8590426 (3)	total: 184ms	remaining: 45.9s
4:	learn: 0.8459663	test: 0.8572695	best: 0.8590426 (3)	total: 199ms	remaining: 39.5s
5:	learn: 0.8490691	test: 0.8581560	best: 0.8590426 (3)	total: 212ms	remaining: 35.2s
6:	learn: 0.8506206	test: 0.8581560	best: 0.8590426 (3)	total: 225ms	remaining: 32s
7:	learn: 0.8515071	test: 0.8563830	best: 0.8590426 (3)	total: 237ms	remaining: 29.4s
8:	learn: 0.8517287	test: 0.8563830	best: 0.8590426 (3)	total: 249ms	remaining: 27.4s
9:	learn: 0.8517287	test: 0.8563830	best: 0.8590426 (3)	total: 264ms	remaining: 26.1s
10:	learn: 0.8517287	test: 0.8563830	best: 0.8590426 (3)	total: 278ms	remaining: 25s
11:	learn: 0.8523936	test

114:	learn: 0.8696809	test: 0.8590426	best: 0.8599291 (79)	total: 978ms	remaining: 7.52s
115:	learn: 0.8696809	test: 0.8590426	best: 0.8599291 (79)	total: 986ms	remaining: 7.51s
116:	learn: 0.8694592	test: 0.8590426	best: 0.8599291 (79)	total: 1000ms	remaining: 7.54s
117:	learn: 0.8692376	test: 0.8599291	best: 0.8599291 (79)	total: 1.01s	remaining: 7.54s
118:	learn: 0.8694592	test: 0.8599291	best: 0.8599291 (79)	total: 1.02s	remaining: 7.59s
119:	learn: 0.8692376	test: 0.8590426	best: 0.8599291 (79)	total: 1.03s	remaining: 7.58s
120:	learn: 0.8692376	test: 0.8590426	best: 0.8599291 (79)	total: 1.04s	remaining: 7.56s
121:	learn: 0.8699025	test: 0.8599291	best: 0.8599291 (79)	total: 1.05s	remaining: 7.53s
122:	learn: 0.8701241	test: 0.8599291	best: 0.8599291 (79)	total: 1.05s	remaining: 7.5s
123:	learn: 0.8703457	test: 0.8590426	best: 0.8599291 (79)	total: 1.06s	remaining: 7.48s
124:	learn: 0.8701241	test: 0.8581560	best: 0.8599291 (79)	total: 1.06s	remaining: 7.45s
125:	learn: 0.8705674

207:	learn: 0.8765514	test: 0.8590426	best: 0.8599291 (79)	total: 1.58s	remaining: 6.01s
208:	learn: 0.8765514	test: 0.8599291	best: 0.8599291 (79)	total: 1.58s	remaining: 6s
209:	learn: 0.8769947	test: 0.8599291	best: 0.8599291 (79)	total: 1.59s	remaining: 5.99s
210:	learn: 0.8769947	test: 0.8599291	best: 0.8599291 (79)	total: 1.6s	remaining: 5.99s
211:	learn: 0.8772163	test: 0.8617021	best: 0.8617021 (211)	total: 1.61s	remaining: 5.97s
212:	learn: 0.8772163	test: 0.8617021	best: 0.8617021 (211)	total: 1.61s	remaining: 5.96s
213:	learn: 0.8776596	test: 0.8599291	best: 0.8617021 (211)	total: 1.62s	remaining: 5.95s
214:	learn: 0.8778812	test: 0.8599291	best: 0.8617021 (211)	total: 1.63s	remaining: 5.94s
215:	learn: 0.8778812	test: 0.8599291	best: 0.8617021 (211)	total: 1.63s	remaining: 5.92s
216:	learn: 0.8781028	test: 0.8599291	best: 0.8617021 (211)	total: 1.64s	remaining: 5.91s
217:	learn: 0.8783245	test: 0.8599291	best: 0.8617021 (211)	total: 1.64s	remaining: 5.9s
218:	learn: 0.87832

304:	learn: 0.8838652	test: 0.8634752	best: 0.8634752 (296)	total: 2.17s	remaining: 4.96s
305:	learn: 0.8838652	test: 0.8634752	best: 0.8634752 (296)	total: 2.18s	remaining: 4.95s
306:	learn: 0.8838652	test: 0.8634752	best: 0.8634752 (296)	total: 2.19s	remaining: 4.94s
307:	learn: 0.8840869	test: 0.8634752	best: 0.8634752 (296)	total: 2.2s	remaining: 4.93s
308:	learn: 0.8840869	test: 0.8634752	best: 0.8634752 (296)	total: 2.21s	remaining: 4.93s
309:	learn: 0.8843085	test: 0.8634752	best: 0.8634752 (296)	total: 2.21s	remaining: 4.92s
310:	learn: 0.8845301	test: 0.8634752	best: 0.8634752 (296)	total: 2.22s	remaining: 4.91s
311:	learn: 0.8845301	test: 0.8634752	best: 0.8634752 (296)	total: 2.23s	remaining: 4.91s
312:	learn: 0.8843085	test: 0.8634752	best: 0.8634752 (296)	total: 2.23s	remaining: 4.9s
313:	learn: 0.8847518	test: 0.8634752	best: 0.8634752 (296)	total: 2.24s	remaining: 4.89s
314:	learn: 0.8847518	test: 0.8634752	best: 0.8634752 (296)	total: 2.24s	remaining: 4.88s
315:	learn: 

396:	learn: 0.8889628	test: 0.8661348	best: 0.8661348 (376)	total: 2.77s	remaining: 4.21s
397:	learn: 0.8889628	test: 0.8661348	best: 0.8661348 (376)	total: 2.78s	remaining: 4.2s
398:	learn: 0.8887411	test: 0.8661348	best: 0.8661348 (376)	total: 2.79s	remaining: 4.19s
399:	learn: 0.8891844	test: 0.8661348	best: 0.8661348 (376)	total: 2.8s	remaining: 4.2s
400:	learn: 0.8891844	test: 0.8661348	best: 0.8661348 (376)	total: 2.81s	remaining: 4.19s
401:	learn: 0.8891844	test: 0.8661348	best: 0.8661348 (376)	total: 2.81s	remaining: 4.18s
402:	learn: 0.8889628	test: 0.8652482	best: 0.8661348 (376)	total: 2.82s	remaining: 4.18s
403:	learn: 0.8889628	test: 0.8652482	best: 0.8661348 (376)	total: 2.83s	remaining: 4.17s
404:	learn: 0.8889628	test: 0.8652482	best: 0.8661348 (376)	total: 2.83s	remaining: 4.16s
405:	learn: 0.8889628	test: 0.8652482	best: 0.8661348 (376)	total: 2.84s	remaining: 4.15s
406:	learn: 0.8889628	test: 0.8652482	best: 0.8661348 (376)	total: 2.84s	remaining: 4.14s
407:	learn: 0

492:	learn: 0.8925089	test: 0.8643617	best: 0.8661348 (376)	total: 3.37s	remaining: 3.46s
493:	learn: 0.8925089	test: 0.8643617	best: 0.8661348 (376)	total: 3.38s	remaining: 3.46s
494:	learn: 0.8925089	test: 0.8643617	best: 0.8661348 (376)	total: 3.39s	remaining: 3.46s
495:	learn: 0.8925089	test: 0.8643617	best: 0.8661348 (376)	total: 3.4s	remaining: 3.45s
496:	learn: 0.8925089	test: 0.8643617	best: 0.8661348 (376)	total: 3.41s	remaining: 3.45s
497:	learn: 0.8925089	test: 0.8643617	best: 0.8661348 (376)	total: 3.42s	remaining: 3.45s
498:	learn: 0.8925089	test: 0.8643617	best: 0.8661348 (376)	total: 3.43s	remaining: 3.45s
499:	learn: 0.8925089	test: 0.8643617	best: 0.8661348 (376)	total: 3.45s	remaining: 3.45s
500:	learn: 0.8925089	test: 0.8643617	best: 0.8661348 (376)	total: 3.46s	remaining: 3.45s
501:	learn: 0.8925089	test: 0.8643617	best: 0.8661348 (376)	total: 3.48s	remaining: 3.45s
502:	learn: 0.8925089	test: 0.8643617	best: 0.8661348 (376)	total: 3.49s	remaining: 3.45s
503:	learn:

599:	learn: 0.8967199	test: 0.8634752	best: 0.8661348 (376)	total: 4.17s	remaining: 2.78s
600:	learn: 0.8967199	test: 0.8634752	best: 0.8661348 (376)	total: 4.18s	remaining: 2.78s
601:	learn: 0.8967199	test: 0.8634752	best: 0.8661348 (376)	total: 4.19s	remaining: 2.77s
602:	learn: 0.8967199	test: 0.8634752	best: 0.8661348 (376)	total: 4.19s	remaining: 2.76s
603:	learn: 0.8967199	test: 0.8634752	best: 0.8661348 (376)	total: 4.2s	remaining: 2.76s
604:	learn: 0.8967199	test: 0.8634752	best: 0.8661348 (376)	total: 4.21s	remaining: 2.75s
605:	learn: 0.8969415	test: 0.8634752	best: 0.8661348 (376)	total: 4.22s	remaining: 2.75s
606:	learn: 0.8969415	test: 0.8634752	best: 0.8661348 (376)	total: 4.23s	remaining: 2.74s
607:	learn: 0.8971631	test: 0.8634752	best: 0.8661348 (376)	total: 4.24s	remaining: 2.73s
608:	learn: 0.8971631	test: 0.8634752	best: 0.8661348 (376)	total: 4.24s	remaining: 2.72s
609:	learn: 0.8971631	test: 0.8634752	best: 0.8661348 (376)	total: 4.25s	remaining: 2.72s
610:	learn:

693:	learn: 0.8993794	test: 0.8652482	best: 0.8661348 (376)	total: 4.79s	remaining: 2.11s
694:	learn: 0.8996011	test: 0.8652482	best: 0.8661348 (376)	total: 4.8s	remaining: 2.11s
695:	learn: 0.8996011	test: 0.8652482	best: 0.8661348 (376)	total: 4.81s	remaining: 2.1s
696:	learn: 0.8998227	test: 0.8652482	best: 0.8661348 (376)	total: 4.82s	remaining: 2.09s
697:	learn: 0.8998227	test: 0.8652482	best: 0.8661348 (376)	total: 4.83s	remaining: 2.09s
698:	learn: 0.8998227	test: 0.8652482	best: 0.8661348 (376)	total: 4.83s	remaining: 2.08s
699:	learn: 0.8998227	test: 0.8652482	best: 0.8661348 (376)	total: 4.84s	remaining: 2.07s
700:	learn: 0.9000443	test: 0.8661348	best: 0.8661348 (376)	total: 4.84s	remaining: 2.07s
701:	learn: 0.9000443	test: 0.8661348	best: 0.8661348 (376)	total: 4.85s	remaining: 2.06s
702:	learn: 0.9000443	test: 0.8661348	best: 0.8661348 (376)	total: 4.86s	remaining: 2.05s
703:	learn: 0.8998227	test: 0.8661348	best: 0.8661348 (376)	total: 4.86s	remaining: 2.04s
704:	learn: 

813:	learn: 0.9022606	test: 0.8625887	best: 0.8661348 (376)	total: 5.59s	remaining: 1.28s
814:	learn: 0.9022606	test: 0.8625887	best: 0.8661348 (376)	total: 5.6s	remaining: 1.27s
815:	learn: 0.9022606	test: 0.8625887	best: 0.8661348 (376)	total: 5.61s	remaining: 1.26s
816:	learn: 0.9022606	test: 0.8625887	best: 0.8661348 (376)	total: 5.62s	remaining: 1.26s
817:	learn: 0.9022606	test: 0.8625887	best: 0.8661348 (376)	total: 5.63s	remaining: 1.25s
818:	learn: 0.9022606	test: 0.8625887	best: 0.8661348 (376)	total: 5.63s	remaining: 1.24s
819:	learn: 0.9022606	test: 0.8625887	best: 0.8661348 (376)	total: 5.64s	remaining: 1.24s
820:	learn: 0.9024823	test: 0.8625887	best: 0.8661348 (376)	total: 5.64s	remaining: 1.23s
821:	learn: 0.9024823	test: 0.8625887	best: 0.8661348 (376)	total: 5.65s	remaining: 1.22s
822:	learn: 0.9024823	test: 0.8625887	best: 0.8661348 (376)	total: 5.66s	remaining: 1.22s
823:	learn: 0.9024823	test: 0.8625887	best: 0.8661348 (376)	total: 5.66s	remaining: 1.21s
824:	learn:

932:	learn: 0.9049202	test: 0.8599291	best: 0.8661348 (376)	total: 6.4s	remaining: 459ms
933:	learn: 0.9049202	test: 0.8599291	best: 0.8661348 (376)	total: 6.4s	remaining: 453ms
934:	learn: 0.9049202	test: 0.8599291	best: 0.8661348 (376)	total: 6.41s	remaining: 446ms
935:	learn: 0.9049202	test: 0.8599291	best: 0.8661348 (376)	total: 6.42s	remaining: 439ms
936:	learn: 0.9049202	test: 0.8599291	best: 0.8661348 (376)	total: 6.43s	remaining: 432ms
937:	learn: 0.9049202	test: 0.8599291	best: 0.8661348 (376)	total: 6.43s	remaining: 425ms
938:	learn: 0.9049202	test: 0.8599291	best: 0.8661348 (376)	total: 6.44s	remaining: 418ms
939:	learn: 0.9051418	test: 0.8599291	best: 0.8661348 (376)	total: 6.45s	remaining: 411ms
940:	learn: 0.9051418	test: 0.8599291	best: 0.8661348 (376)	total: 6.46s	remaining: 405ms
941:	learn: 0.9051418	test: 0.8599291	best: 0.8661348 (376)	total: 6.47s	remaining: 398ms
942:	learn: 0.9051418	test: 0.8599291	best: 0.8661348 (376)	total: 6.47s	remaining: 391ms
943:	learn: 

In [18]:
y_pred = model.predict_proba(X_test_new)
y_pred

array([[0.00122966, 0.99877034],
       [0.49310915, 0.50689085],
       [0.91725427, 0.08274573],
       ...,
       [0.87601987, 0.12398013],
       [0.84183733, 0.15816267],
       [0.8552997 , 0.1447003 ]])

In [19]:
y_pred = y_pred[:, 1:]

In [20]:
y_pred.shape

(103873, 1)

In [21]:
pred_proba = pd.DataFrame(y_pred, columns = {'proba'})
pred_proba = X_test.loc[:,['user_id', 'item_id']].join(pred_proba)
pred_proba.head(5)

,user_id,item_id,proba
0,166,35,0.998770
1,166,19,0.506891
2,166,145,0.082746
3,166,36,0.844620
4,166,32,0.489218


In [22]:
pred_proba = pred_proba.sort_values(by = ['user_id', 'proba'], ascending=False)

In [23]:
pred_proba.head()

,user_id,item_id,proba
80901,496,76,0.999000
80883,496,35,0.998871
80888,496,37,0.998713
80889,496,22,0.998532
80964,496,60,0.996337


In [24]:
len(test_ids)

497

In [25]:
a = np.zeros(user_features.shape[0]*20).reshape(20, user_features.shape[0])

j=0

for user in test_ids:
    item = pred_proba[pred_proba['user_id'] == user].nlargest(20, columns = 'proba').reset_index(drop = True)
    for i in range(20):
        x = item.iloc[i]
        a[i][j] = x.item_id
    j+=1

In [35]:
d = {'user_id' : test_ids}
a = a.astype(int)
ans = pd.DataFrame(data = d)
for i in range(20):
    ans[str(i)] = a[i]
ans = ans.values[:,1:]
ans

array([[ 76,  35,  37, ...,   5, 147,   1],
       [ 76,  35,  37, ...,  36,  34, 147],
       [ 76,  35,  37, ...,  33,  32,  80],
       ...,
       [ 76,  35,  37, ...,  12,  42,  68],
       [ 76,  35,  37, ...,  67,   7,  66],
       [ 76,  35,  37, ..., 126, 136,  36]])

In [42]:
counter = 0
test_ids = test['user_id'].tolist()
def full_pred(pred, liked):
    global counter
    counter += 1
    j = 0
    most_liked = [76,35,37,22,60,65,80,72,66,200,58, 7,11,40,148,36,21,5,59,87]
    while len(pred) < 20:
        if most_liked[j] not in liked:
            pred.append(most_liked[j])
        else:
            j+=1
        j+=1
    return pred

a = []
k = 0
for pred in ans:
    pred = list(pred)
    user_x = test_ids[k]
    liked = [] 
    for i in range(len(interactions[user_x])):
        if int(interactions[user_x][i]) == -1 and i in pred:
            pred.remove(i)
        if int(interactions[user_x][i]) == 1:
            if i in pred:
                pred.remove(i)
            liked.append(i)
    if len(pred) < 20:       
        #print(user_x)
        pred = full_pred(pred, liked)
    k+=1
    
    top_20_items = pred # [k for k, v in sorted_]
    a.append([user_x] + top_20_items[:20])

In [43]:
a = np.array(a)
cols = ['user_id'] + [str(i) for i in range(20)]
ans = pd.DataFrame(data = a, columns = cols)
ans.head()

,user_id,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,166,76,35,37,22,60,65,72,80,66,...,58,7,40,11,148,36,21,5,147,1
1,26,76,35,37,146,87,22,67,33,21,...,49,32,19,78,88,59,36,34,147,59
2,41,76,35,37,22,60,65,21,5,36,...,148,146,87,67,40,11,33,32,80,59
3,286,76,35,37,22,60,21,65,5,36,...,148,146,87,67,7,11,33,32,80,59
4,108,76,35,37,22,60,21,5,36,59,...,146,87,67,7,40,11,33,32,80,59


In [44]:
ans.to_csv('ans_cat.csv', index = False)